## Why Snapshots?

I need to do some testing which requires a lot of setup, and I want to be able to quickly revert my changes. I could do a backup & restore, but I don't really have space for all that, and it takes a long time to do those restores because of the size of the database.

https://docs.microsoft.com/en-us/sql/relational-databases/databases/database-snapshots-sql-server?view=sql-server-ver15  

https://docs.microsoft.com/en-us/sql/relational-databases/databases/create-a-database-snapshot-transact-sql?view=sql-server-ver15

## Let's Make a Snapshot!

In [13]:
create database StackOverflow_Snapshot on 
    (name=StackOverflow_1
    , Filename='C:\SQL\Data\SQL17\StackOverflow1_Snapshot.ss')
    , (name=StackOverflow_2
    , Filename='C:\SQL\Data\SQL17\StackOverflow2_Snapshot.ss')
    , (name=StackOverflow_3
    , Filename='C:\SQL\Data\SQL17\StackOverflow3_Snapshot.ss')
    , (name=StackOverflow_4
    , Filename='C:\SQL\Data\SQL17\StackOverflow4_Snapshot.ss')
 as SNAPSHOT of StackOverflow;

Commands completed successfully.

Total execution time: 00:00:00.285

## That Was Quick

Did it work? Let's find out

In [14]:
select * from sys.databases where source_database_id is not null;
select s.name as SourceDB, t.name as SnapshotDB from sys.databases s join sys.databases t on s.database_id = t.source_database_id

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.283

name,database_id,source_database_id,owner_sid,create_date,compatibility_level,collation_name,user_access,user_access_desc,is_read_only,is_auto_close_on,is_auto_shrink_on,state,state_desc,is_in_standby,is_cleanly_shutdown,is_supplemental_logging_enabled,snapshot_isolation_state,snapshot_isolation_state_desc,is_read_committed_snapshot_on,recovery_model,recovery_model_desc,page_verify_option,page_verify_option_desc,is_auto_create_stats_on,is_auto_create_stats_incremental_on,is_auto_update_stats_on,is_auto_update_stats_async_on,is_ansi_null_default_on,is_ansi_nulls_on,is_ansi_padding_on,is_ansi_warnings_on,is_arithabort_on,is_concat_null_yields_null_on,is_numeric_roundabort_on,is_quoted_identifier_on,is_recursive_triggers_on,is_cursor_close_on_commit_on,is_local_cursor_default,is_fulltext_enabled,is_trustworthy_on,is_db_chaining_on,is_parameterization_forced,is_master_key_encrypted_by_server,is_query_store_on,is_published,is_subscribed,is_merge_published,is_distributor,is_sync_with_backup,service_broker_guid,is_broker_enabled,log_reuse_wait,log_reuse_wait_desc,is_date_correlation_on,is_cdc_enabled,is_encrypted,is_honor_broker_priority_on,replica_id,group_database_id,resource_pool_id,default_language_lcid,default_language_name,default_fulltext_language_lcid,default_fulltext_language_name,is_nested_triggers_on,is_transform_noise_words_on,two_digit_year_cutoff,containment,containment_desc,target_recovery_time_in_seconds,delayed_durability,delayed_durability_desc,is_memory_optimized_elevate_to_snapshot_on,is_federation_member,is_remote_data_archive_enabled,is_mixed_page_allocation_on,is_temporal_history_retention_enabled
StackOverflow_Snapshot,23,21,0x01,2020-10-18 15:15:32.853,100,SQL_Latin1_General_CP1_CI_AS,0,MULTI_USER,1,0,0,0,ONLINE,0,1,0,1,ON,0,3,SIMPLE,2,CHECKSUM,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9e22b50c-2f4a-4625-a73a-6694d4cfb806,0,1,CHECKPOINT,0,0,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0,NONE,0,0,DISABLED,0,0,0,0,1


SourceDB,SnapshotDB
StackOverflow,StackOverflow_Snapshot


## Time to Test

Where do I fall in Stack Overflow rankings?

In [15]:
select * from (
    select accountid,displayname, [Location], reputation,upvotes,downvotes, creationdate from StackOverflow.dbo.users where DisplayName= 'alroc'
    UNION
    select top 5 accountid,displayname, [Location], reputation,upvotes,downvotes, creationdate from StackOverflow.dbo.Users order by reputation desc
) as UserRep order by reputation desc;

(7 rows affected)

Total execution time: 00:00:06.519

accountid,displayname,Location,reputation,upvotes,downvotes,creationdate
11683,Jon Skeet,"Reading, United Kingdom",1147559,16489,6951,2008-09-26 12:05:05.150
4243,VonC,France,910732,51729,274,2008-09-13 22:22:33.173
52822,BalusC,"Willemstad, Curaçao",900136,15394,21418,2009-08-17 16:42:02.403
14332,Darin Dimitrov,"Sofia, Bulgaria",887923,1954,2651,2008-10-19 16:07:47.823
1165580,Gordon Linoff,"New York, United States",886150,14639,41,2012-01-11 19:53:57.590
1394230,alroc,United States,24229,933,289,2012-04-10 14:51:44.847
2122467,Alroc,NULL,127,15,0,2012-12-07 12:34:10.123


I don't like where I'm sitting, let's give it a little boost.

In [16]:
Update StackOverflow.dbo.Users
set reputation = (select reputation - 1 from StackOverflow.dbo.Users where accountid = 1165580);

select * from (
    select accountid,displayname, [Location], reputation,upvotes,downvotes, creationdate from StackOverflow.dbo.users where DisplayName= 'alroc'
    UNION
    select top 5 accountid,displayname, [Location], reputation,upvotes,downvotes, creationdate from StackOverflow.dbo.Users order by reputation desc
) as UserRep order by reputation desc;

(11376305 rows affected)

(7 rows affected)

Total execution time: 00:05:15.424

accountid,displayname,Location,reputation,upvotes,downvotes,creationdate
19120,netic,Australia,886149,24,0,2008-12-09 08:32:03.307
19121,Julian Popov,Bulgaria,886149,152,3,2008-12-09 08:53:51.420
19122,user44538,NULL,886149,31,0,2008-12-09 09:05:46.827
19123,si618,"Adelaide, South Australia",886149,1778,24,2008-12-09 09:12:48.373
19124,lakshmanaraj,India,886149,0,0,2008-12-09 09:19:24.770
1394230,alroc,United States,886149,933,289,2012-04-10 14:51:44.847
2122467,Alroc,NULL,886149,15,0,2012-12-07 12:34:10.123


## WHOOPS!

That's not what I wanted. Let's roll it back and try again.

In [21]:
Restore database StackOverflow from database_SNAPSHOT = 'StackOverflow_Snapshot';

Commands completed successfully.

Total execution time: 00:00:39.458

In [20]:
Update StackOverflow.dbo.Users
set reputation = (select reputation -1 from StackOverflow.dbo.Users where accountid = 1165580)
where accountid = 1394230;

select * from (
    select accountid,displayname, [Location], reputation,upvotes,downvotes, creationdate from StackOverflow.dbo.users where DisplayName= 'alroc'
    UNION
    select top 5 accountid,displayname, [Location], reputation,upvotes,downvotes, creationdate from StackOverflow.dbo.Users order by reputation desc
) as UserRep order by reputation desc;

(1 row affected)

(7 rows affected)

Total execution time: 00:00:11.409

accountid,displayname,Location,reputation,upvotes,downvotes,creationdate
11683,Jon Skeet,"Reading, United Kingdom",1147559,16489,6951,2008-09-26 12:05:05.150
4243,VonC,France,910732,51729,274,2008-09-13 22:22:33.173
52822,BalusC,"Willemstad, Curaçao",900136,15394,21418,2009-08-17 16:42:02.403
14332,Darin Dimitrov,"Sofia, Bulgaria",887923,1954,2651,2008-10-19 16:07:47.823
1165580,Gordon Linoff,"New York, United States",886150,14639,41,2012-01-11 19:53:57.590
1394230,alroc,United States,886149,933,289,2012-04-10 14:51:44.847
2122467,Alroc,NULL,127,15,0,2012-12-07 12:34:10.123


## That's much better!

Let's stick with this and remove the snapshot now

In [22]:
drop database StackOverflow_Snapshot;
select * from sys.databases where source_database_id is not null;
select s.name as SourceDB, t.name as SnapshotDB from sys.databases s join sys.databases t on s.database_id = t.source_database_id

(0 rows affected)

(0 rows affected)

Total execution time: 00:00:00.807

name,database_id,source_database_id,owner_sid,create_date,compatibility_level,collation_name,user_access,user_access_desc,is_read_only,is_auto_close_on,is_auto_shrink_on,state,state_desc,is_in_standby,is_cleanly_shutdown,is_supplemental_logging_enabled,snapshot_isolation_state,snapshot_isolation_state_desc,is_read_committed_snapshot_on,recovery_model,recovery_model_desc,page_verify_option,page_verify_option_desc,is_auto_create_stats_on,is_auto_create_stats_incremental_on,is_auto_update_stats_on,is_auto_update_stats_async_on,is_ansi_null_default_on,is_ansi_nulls_on,is_ansi_padding_on,is_ansi_warnings_on,is_arithabort_on,is_concat_null_yields_null_on,is_numeric_roundabort_on,is_quoted_identifier_on,is_recursive_triggers_on,is_cursor_close_on_commit_on,is_local_cursor_default,is_fulltext_enabled,is_trustworthy_on,is_db_chaining_on,is_parameterization_forced,is_master_key_encrypted_by_server,is_query_store_on,is_published,is_subscribed,is_merge_published,is_distributor,is_sync_with_backup,service_broker_guid,is_broker_enabled,log_reuse_wait,log_reuse_wait_desc,is_date_correlation_on,is_cdc_enabled,is_encrypted,is_honor_broker_priority_on,replica_id,group_database_id,resource_pool_id,default_language_lcid,default_language_name,default_fulltext_language_lcid,default_fulltext_language_name,is_nested_triggers_on,is_transform_noise_words_on,two_digit_year_cutoff,containment,containment_desc,target_recovery_time_in_seconds,delayed_durability,delayed_durability_desc,is_memory_optimized_elevate_to_snapshot_on,is_federation_member,is_remote_data_archive_enabled,is_mixed_page_allocation_on,is_temporal_history_retention_enabled


SourceDB,SnapshotDB
